In [25]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import xlrd
import pickle
import numpy as np
import os

# Kinect3D

In [3]:

# 读取数据
file_path = '/Dataset/Kinect3D.csv'
data = pd.read_csv(file_path)

# 定义关节名称
joints = ["Head", "c7","SpineShoulder", "Spine", "Sacr", "LShoulder", "LElbow", "LWrist", "LHand", "RShoulder",
          "RElbow", "RWrist", "RHand", "LHip", "LKnee", "LAnkle", "LFoot",
          "RHip", "RKnee", "RAnkle", "RFoot", "LHandTip", "LThumb", "RHandTip", "RThumb"]
#sacr骶骨,hip臀髋

# 定义关节间的连接
edges = [("Head", "c7"),("c7", "SpineShoulder"),("SpineShoulder","Spine"),("Spine", "Sacr"),
         ("SpineShoulder", "LShoulder"), ("LShoulder", "LElbow"), ("LElbow", "LWrist"), ("LWrist", "LHand"), ("LHand", "LHandTip"), ("LHand", "LThumb"),
         ("SpineShoulder", "RShoulder"), ("RShoulder", "RElbow"), ("RElbow", "RWrist"), ("RWrist", "RHand"), ("RHand", "RHandTip"), ("RHand", "RThumb"),
         ("Sacr", "LHip"), ("LHip", "LKnee"), ("LKnee", "LAnkle"), ("LAnkle", "LFoot"),
         ("Sacr", "RHip"), ("RHip", "RKnee"), ("RKnee", "RAnkle"), ("RAnkle", "RFoot")]

# 计算两点间的欧几里得距离
def euclidean_distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))

# 构建一个邻接矩阵
def build_adjacency_matrix(frame_data):
    n = len(joints)
    adjacency_matrix = np.zeros((n, n))

    # 填充邻接矩阵
    for edge in edges:
        idx1 = joints.index(edge[0])
        idx2 = joints.index(edge[1])
        joint1 = frame_data[[f"{edge[0]}_x", f"{edge[0]}_y", f"{edge[0]}_z"]].values
        joint2 = frame_data[[f"{edge[1]}_x", f"{edge[1]}_y", f"{edge[1]}_z"]].values
        distance = euclidean_distance(joint1, joint2)
        adjacency_matrix[idx1, idx2] = 1 / distance if distance != 0 else 0
        adjacency_matrix[idx2, idx1] = adjacency_matrix[idx1, idx2]

    return adjacency_matrix

# 存储图的列表
graph_list = []

# 遍历每一帧
for index, frame_data in data.iterrows():
    # 创建有向图
    G = nx.DiGraph()

    # 添加节点和边
    for joint in joints:
        x, y, z = frame_data[[f"{joint}_x", f"{joint}_y", f"{joint}_z"]]
        G.add_node(joint, pos=(x, y))
    for edge in edges:
        G.add_edge(*edge)

    # 添加带有权重的边
    adjacency_matrix = build_adjacency_matrix(frame_data)
    for i in range(adjacency_matrix.shape[0]):
        for j in range(adjacency_matrix.shape[1]):
            if adjacency_matrix[i, j] != 0:
                G.add_edge(joints[i], joints[j], weight=adjacency_matrix[i, j])

    # 将图添加到列表中
    graph_list.append(G)

# 定义pkl文件路径
desktop_path = '/Dataset/'
pkl_file_path = desktop_path + 'Kinect3D.pkl'

# 保存图的列表到桌面的pkl文件
with open(pkl_file_path, 'wb') as file:
    pickle.dump(graph_list, file)

print(f'Graph list saved to: {pkl_file_path}')


Graph list saved to: /Dataset/Kinect3D.pkl


In [10]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import os

# 读取数据
file_path = '/Dataset/Kinect3D.csv'
data = pd.read_csv(file_path)

# 定义关节名称
joints = ["Head", "c7","SpineShoulder", "Spine", "Sacr", "LShoulder", "LElbow", "LWrist", "LHand", "RShoulder",
          "RElbow", "RWrist", "RHand", "LHip", "LKnee", "LAnkle", "LFoot",
          "RHip", "RKnee", "RAnkle", "RFoot", "LHandTip", "LThumb", "RHandTip", "RThumb"]
#sacr骶骨,hip臀髋

# 定义关节间的连接
edges = [("Head", "c7"),("c7", "SpineShoulder"),("SpineShoulder","Spine"),("Spine", "Sacr"),
         ("SpineShoulder", "LShoulder"), ("LShoulder", "LElbow"), ("LElbow", "LWrist"), ("LWrist", "LHand"), ("LHand", "LHandTip"), ("LHand", "LThumb"),
         ("SpineShoulder", "RShoulder"), ("RShoulder", "RElbow"), ("RElbow", "RWrist"), ("RWrist", "RHand"), ("RHand", "RHandTip"), ("RHand", "RThumb"),
         ("Sacr", "LHip"), ("LHip", "LKnee"), ("LKnee", "LAnkle"), ("LAnkle", "LFoot"),
         ("Sacr", "RHip"), ("RHip", "RKnee"), ("RKnee", "RAnkle"), ("RAnkle", "RFoot")]

# 创建一个文件夹来保存可视化图像
output_folder = '/img_Kinect3D'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 选择前328帧进行可视化
for index in range(328):
    frame_data = data.iloc[index]

    # 创建图
    G = nx.DiGraph()

    # 添加节点和边
    for joint in joints:
        x, y, z = frame_data[[f"{joint}_x", f"{joint}_y", f"{joint}_z"]]
        G.add_node(joint, pos=(x, y))
    for edge in edges:
        G.add_edge(*edge)
    # 添加带有权重的边
    adjacency_matrix = build_adjacency_matrix(frame_data)
    for i in range(adjacency_matrix.shape[0]):
        for j in range(adjacency_matrix.shape[1]):
            if adjacency_matrix[i, j] != 0:
                G.add_edge(joints[i], joints[j], weight=adjacency_matrix[i, j])

    # 获取节点的位置
    pos = nx.get_node_attributes(G, 'pos')

    # 绘制图
    plt.figure(figsize=(8, 12))
    nx.draw(G, pos, with_labels=True, node_size=700, node_color="skyblue", font_size=10)
    plt.title(f"Node Graph Visualization - Frame {index}")

    # 保存图像
    plt.savefig(os.path.join(output_folder, f'frame_{index}.png'))
    plt.close()


# AlphaPose

In [ ]:
# 读取数据
file_path = '/Dataset/AlphaPose.csv'
data = pd.read_csv(file_path)


# 定义关节名称
joints = ["Nose", "LEye", "REye", "LShoulder", "LElbow", "LWrist", "RShoulder",
          "RElbow", "RWrist", "LHip", "LKnee", "LAnkle", "RHip", "RKnee", "RAnkle"]

# 定义关节间的连接
edges = [("LShoulder", "LHip"), ("LHip", "LKnee"), ("LKnee", "LAnkle"), 
         ("RShoulder", "RHip"), ("RHip", "RKnee"), ("RKnee", "RAnkle"),
         ("Nose", "LEye"), ("Nose", "REye"),("LHip","RHip"),
         ("Nose", "LShoulder"), ("LShoulder", "LElbow"), ("LElbow", "LWrist"), 
         ("Nose", "RShoulder"), ("RShoulder", "RElbow"), ("RElbow", "RWrist")]

# 计算两点间的欧几里得距离
def euclidean_distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))

# 构建一个邻接矩阵
def build_adjacency_matrix(frame_data):
    n = len(joints)
    adjacency_matrix = np.zeros((n, n))

    # 填充邻接矩阵
    for edge in edges:
        idx1 = joints.index(edge[0])
        idx2 = joints.index(edge[1])
        joint1 = frame_data[[f"{edge[0]}_x", f"{edge[0]}_y"]].values
        joint2 = frame_data[[f"{edge[1]}_x", f"{edge[1]}_y"]].values
        distance = euclidean_distance(joint1, joint2)
        adjacency_matrix[idx1, idx2] = 1 / distance if distance != 0 else 0
        adjacency_matrix[idx2, idx1] = adjacency_matrix[idx1, idx2]  # 无向图

    return adjacency_matrix

# 存储图的列表
graph_list = []

# 遍历每一帧
for index, frame_data in data.iterrows():
    # 创建有向图
    G = nx.DiGraph()

    # 添加节点和边
    for joint in joints:
        x, y = frame_data[[f"{joint}_x", f"{joint}_y"]]
        G.add_node(joint, pos=(x, y))
    for edge in edges:
        G.add_edge(*edge)

    # 添加带有权重的边
    adjacency_matrix = build_adjacency_matrix(frame_data)
    for i in range(adjacency_matrix.shape[0]):
        for j in range(adjacency_matrix.shape[1]):
            if adjacency_matrix[i, j] != 0:
                G.add_edge(joints[i], joints[j], weight=adjacency_matrix[i, j])

    # 将图添加到列表中
    graph_list.append(G)

# 定义pkl文件路径
desktop_path = '/Dataset/'
pkl_file_path = desktop_path + 'AlphaPose.pkl'

# 保存图的列表到桌面的pkl文件
with open(pkl_file_path, 'wb') as file:
    pickle.dump(graph_list, file)

print(f'Graph list saved to: {pkl_file_path}')


In [ ]:

# 读取数据
file_path = '/Dataset/AlphaPose.csv'
data = pd.read_csv(file_path)

# 定义关节名称
joints = ["Nose", "LEye", "REye", "LShoulder", "LElbow", "LWrist", "RShoulder",
          "RElbow", "RWrist", "LHip", "LKnee", "LAnkle", "RHip", "RKnee", "RAnkle"]

# 定义关节间的连接
edges = [("LShoulder", "LHip"), ("LHip", "LKnee"), ("LKnee", "LAnkle"), 
         ("RShoulder", "RHip"), ("RHip", "RKnee"), ("RKnee", "RAnkle"),
         ("Nose", "LEye"), ("Nose", "REye"),("LHip","RHip"),("LShoulder","RShoulder"),
         ("Nose", "LShoulder"), ("LShoulder", "LElbow"), ("LElbow", "LWrist"), 
         ("Nose", "RShoulder"), ("RShoulder", "RElbow"), ("RElbow", "RWrist")]

# 创建一个文件夹来保存可视化图像
output_folder = '/img_AlphaPose'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 选择前875帧进行可视化
for index in range(875):
    frame_data = data.iloc[index]

    # 创建图
    G = nx.DiGraph()

    # 添加节点和边
    for joint in joints:
        x, y = frame_data[[f"{joint}_x", f"{joint}_y"]]
        G.add_node(joint, pos=(x, y))
    for edge in edges:
        G.add_edge(*edge)
    # 添加带有权重的边
    adjacency_matrix = build_adjacency_matrix(frame_data)
    for i in range(adjacency_matrix.shape[0]):
        for j in range(adjacency_matrix.shape[1]):
            if adjacency_matrix[i, j] != 0:
                G.add_edge(joints[i], joints[j], weight=adjacency_matrix[i, j])
    # 获取节点的位置
    pos = nx.get_node_attributes(G, 'pos')
    
    # 定义旋转矩阵
    theta = np.radians(165)  # 旋转角度，这里是 165 度
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])

    # 应用旋转矩阵到节点位置
    rotated_node_positions = {k: np.dot(rotation_matrix, np.array([x, y])) for k, (x, y) in pos.items()}

    # 绘制图
    plt.figure(figsize=(8, 12))#原始图片是（8，12）
    nx.draw(G, rotated_node_positions, with_labels=True, node_size=700, node_color="skyblue", font_size=10)
    plt.title(f"Node Graph Visualization - Frame {index}")
   
    # 显示图形
    #plt.show()
    # 保存图像
    plt.savefig(os.path.join(output_folder, f'frame_{index}.png'))
    plt.close()